In [1]:
import numpy as np
import pandas as pd
import acquire as a
import requests
from bs4 import BeautifulSoup
import re

In [2]:
data = a.scrape_github_data()

#### By adding repo names to the REPOS area of the acquire.py file, in the format 'gocodeup/codeup-setup-script' (no spaces) they will be added to the list of repo readme contents that will be scraped.

In [3]:
data

[{'repo': 'gocodeup/codeup-setup-script',
  'language': 'Shell',
  'readme_contents': '# Codeup Setup Script\n\nSetup script for Codeup students\' laptops to install all of the tools we will\nneed for the java course. We will install the following:\n\n- [xcode](https://developer.apple.com/xcode/features/): command line tools for\n  macs\n- [brew](http://brew.sh/): package manager for macs\n- [java](https://en.wikipedia.org/wiki/Java_(programming_language))\n- [tomcat](http://tomcat.apache.org/): the java webserver\n- [maven](https://maven.apache.org/): a java dependency and build management tool\n- [mysql](https://www.mysql.com/): the database we\'ll use for the class\n- [node js](https://nodejs.org/en/): a JavaScript runtime outside the browser\n- [npm](https://www.npmjs.com/): a package manager for JavaScript\n- [intellij](https://www.jetbrains.com/idea/): a Java IDE\n\nIn addition, we will:\n\n- setup ssh keys for the student\'s laptop and guide them through the process of\n  linkin

#### Our first step should be to scrape a list of repo names. On github the repo names have spaces, so the spaces will also need to be removed before they can be added to the REPOS list.

In [4]:
# nlp projects using python
python_url = 'https://github.com/topics/nlp?l=python'
# nlp projects using html
html_url = 'https://github.com/topics/nlp?l=html'

In [29]:
response = requests.get('https://github.com/topics/nlp?l=python')

In [6]:
response

<Response [200]>

In [7]:
soup = BeautifulSoup(response.content)

In [8]:
# soup

In [9]:
# grab one project repo name from the soup
soup.find_all('a', class_="text-bold wb-break-word")[0]['href']

'/huggingface/transformers'

In [10]:
len(soup.find_all('a', class_="text-bold wb-break-word"))

20

In [11]:
python_url_list = [item['href'] for item in 
                   soup.find_all('a', class_="text-bold wb-break-word")]

In [12]:
python_url_list

['/huggingface/transformers',
 '/apachecn/ailearning',
 '/google-research/bert',
 '/hankcs/HanLP',
 '/explosion/spaCy',
 '/RasaHQ/rasa',
 '/huggingface/datasets',
 '/mindsdb/mindsdb',
 '/RaRe-Technologies/gensim',
 '/flairNLP/flair',
 '/microsoft/unilm',
 '/nltk/nltk',
 '/PaddlePaddle/PaddleHub',
 '/allenai/allennlp',
 '/chiphuyen/stanford-tensorflow-tutorials',
 '/PaddlePaddle/PaddleNLP',
 '/deepset-ai/haystack',
 '/sloria/TextBlob',
 '/ymcui/Chinese-BERT-wwm',
 '/ymcui/Chinese-LLaMA-Alpaca']

#### There is 20, we will need to remove the '/' at the begining of the repo name 

In [23]:
python_url_list = [re.sub(r'^/', '',python_url_list[i]) 
                   for i in range(len(python_url_list))]

In [24]:
python_url_list

['huggingface/transformers',
 'apachecn/ailearning',
 'google-research/bert',
 'hankcs/HanLP',
 'explosion/spaCy',
 'RasaHQ/rasa',
 'huggingface/datasets',
 'mindsdb/mindsdb',
 'RaRe-Technologies/gensim',
 'flairNLP/flair',
 'microsoft/unilm',
 'nltk/nltk',
 'PaddlePaddle/PaddleHub',
 'allenai/allennlp',
 'chiphuyen/stanford-tensorflow-tutorials',
 'PaddlePaddle/PaddleNLP',
 'deepset-ai/haystack',
 'sloria/TextBlob',
 'ymcui/Chinese-BERT-wwm',
 'ymcui/Chinese-LLaMA-Alpaca']

### now to figure out how to get more results

In [21]:
#   &page=2

i=2
page_url = f'{python_url}&page={i}'
page_url

'https://github.com/topics/nlp?l=python&page=2'

In [30]:
response2 = requests.get(page_url)

In [31]:
soup2 = BeautifulSoup(response2.content)

In [32]:
python2_url_list = [item['href'] for item in 
                   soup2.find_all('a', class_="text-bold wb-break-word")]

In [33]:
python2_url_list

['/brightmart/text_classification',
 '/jadore801120/attention-is-all-you-need-pytorch',
 '/Morizeyao/GPT2-Chinese',
 '/PaddlePaddle/models',
 '/stanfordnlp/stanza',
 '/NVIDIA/NeMo',
 '/MycroftAI/mycroft-core',
 '/deeppavlov/DeepPavlov',
 '/microsoft/nlp-recipes',
 '/zihangdai/xlnet',
 '/PaddlePaddle/ERNIE',
 '/codertimo/BERT-pytorch',
 '/vi3k6i5/flashtext',
 '/jessevig/bertviz',
 '/chatopera/Synonyms',
 '/HIT-SCIR/ltp',
 '/MaartenGr/BERTopic',
 '/neuml/txtai',
 '/snipsco/snips-nlu',
 '/pytorch/text']

### So we can get 20 urls per page, lets shoot for say, 300 python projects, and 200 html projects (since the project guidelines say to try to get 500). That will be 15 pages of python urls and 10 of html urls

In [58]:
def get_repo_names(url, num_pages):
    url_list = []
    for page_num in range(1, num_pages):
        page_url = f'{url}&page={page_num}'
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content)
        url_list = url_list + [item['href'] for item in 
                         soup.find_all('a', class_="text-bold wb-break-word")]
    return [re.sub(r'^/', '', url_list[i]) for i in range(len(url_list))]

In [63]:
repo_names = get_repo_names(python_url, 16)

In [64]:
repo_names = repo_names + get_repo_names(html_url, 11)

In [65]:
len(repo_names)

500

In [66]:
repo_names

['huggingface/transformers',
 'apachecn/ailearning',
 'google-research/bert',
 'hankcs/HanLP',
 'explosion/spaCy',
 'RasaHQ/rasa',
 'huggingface/datasets',
 'mindsdb/mindsdb',
 'RaRe-Technologies/gensim',
 'flairNLP/flair',
 'microsoft/unilm',
 'nltk/nltk',
 'PaddlePaddle/PaddleHub',
 'allenai/allennlp',
 'chiphuyen/stanford-tensorflow-tutorials',
 'PaddlePaddle/PaddleNLP',
 'deepset-ai/haystack',
 'sloria/TextBlob',
 'ymcui/Chinese-BERT-wwm',
 'ymcui/Chinese-LLaMA-Alpaca',
 'brightmart/text_classification',
 'jadore801120/attention-is-all-you-need-pytorch',
 'Morizeyao/GPT2-Chinese',
 'PaddlePaddle/models',
 'stanfordnlp/stanza',
 'NVIDIA/NeMo',
 'MycroftAI/mycroft-core',
 'deeppavlov/DeepPavlov',
 'microsoft/nlp-recipes',
 'zihangdai/xlnet',
 'PaddlePaddle/ERNIE',
 'codertimo/BERT-pytorch',
 'vi3k6i5/flashtext',
 'jessevig/bertviz',
 'chatopera/Synonyms',
 'HIT-SCIR/ltp',
 'MaartenGr/BERTopic',
 'neuml/txtai',
 'snipsco/snips-nlu',
 'pytorch/text',
 'thunlp/OpenPrompt',
 'miso-belica

## Now we have 500 repo names, some of them are chinese though